# Project Notebook

The main stuff will be done here!

## What is needed?

For this, I will use `mysql`, which I will host locally. This needs to be runned for `pymssql` to work. After that I can make queries with pymssql when I connect to my local hosted server. This makes development easier. As I use a different machine from last time when I used these, I have to install them again.

**Mysql** local install:

1. Download it from their site! 
2. Install the necessary tools! (I did all)
3. ADD IT TO `PATH`
4. Check with `mysql -V` in terminal if it is installed correctly (restart my be needed needed)
5. ENJOY!

`On windows, it creates a very okeyish config file with reasonable variables.`

`On linux, there is no need to be added to path, but the config file is empty.`

**pymssql**:

1. `pip install -u pymsql`

**.ENV**

It is a good practice to create a new user with granted permissions (don't use root), but it's necessary to hide these credentials if the work is shared: create an .env file that has the information and block its upload to github!

**Helpful stuff**

https://www.digitalocean.com/community/tutorials/a-basic-mysql-tutorial

https://www.digitalocean.com/community/tutorials/how-to-create-a-new-user-and-grant-permissions-in-mysql

In [12]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import re

import sys
import os
import pathlib
import glob

import pymssql

import mysql.connector
from mysql.connector import Error
from mysql.connector import OperationalError, ProgrammingError

from dotenv import load_dotenv
load_dotenv()

True

**Usage of .env is essential in hidin credenials**

In [2]:
usr = os.getenv('user')
psswrd = os.getenv('password')

Let's test i this works

In [3]:
#how to use it
"""
# This opens a connection to the MS SQL server
conn = pymssql.connect(user=usr, 
                       password=psswrd,
                       host='localhost',
                       database='sys')

cursor = conn.cursor()
# A test query
res = cursor.execute('select * from shot;')
print(cursor.fetchone())
"""

db_connection = mysql.connector.connect(user=usr,
                                        passwd=psswrd ,
                                        host='localhost',
                                        database = 'sys',
                                        use_pure=True)

db_cursor = db_connection.cursor()
db_cursor.execute('select* from session;')
for db in db_cursor:
    print(db)

db_connection.close()
db_cursor.close()



(49, 8, 'szaboadamg@localhost', 'sys', 'Query', 'executing', 0, 'select* from session', '33.21 ms', None, '1.77 ms', 0, 0, 0, 2, 0, 'YES', None, None, '1.58 MiB', None, None, None, '3.52 ms', 'ACTIVE', 'NO', '17388', None)


True

Now I just need to create this course's database and create the table that has the .csv loaded in...

```
    load data infile 'data\_linklist.csv'
    into table linklist
    fields terminated by ','
    lines terminated by '\n'
    ignore 1 rows;

```

THe problem is that the setup file has 2 rows specifying where the mysql server can take files from. One of them prevents `local` file insertion, the other gives a strict path.

**But this needs a well defined table beforehand.** With these, the first 3 task is complete:

1. Get the data from kaggle
2. Load the tables into a database
3. Create linklist

## 4. Network Visualizations

Visualizing the networks with the nodes and links. Nodes that are more connected shall be bigger.

In [43]:
import networkx as nx

In [44]:
db_connection = mysql.connector.connect(user=usr,
                                        passwd=psswrd ,
                                        host='localhost',
                                        database = 'datamodproj',
                                        use_pure=True)

Llist = []
count=0
db_cursor = db_connection.cursor()
db_cursor.execute('select* from linklist;')
for db in db_cursor:
    Llist.append(db)
    count=count+1

print(count)

db_connection.close()
db_cursor.close()

23036


True

In [45]:
Llist[1]

('a925ecc2-c223-5555-b66e-0b5b24122201',
 ' ee9c03ac-4251-5042-ac85-986db9228a17\r')

It's strange that the `\r` is present in the returned result of the query. Regex can remove, but I have never seen such thing before. Maybe due to the loading?

In [46]:
for i in range(len(Llist)):
    sub = list(Llist[i])
    sub[1] = re.sub(r'\r', '', sub[1]).strip()
    Llist[i] = tuple(sub)
    
Llist_a = np.array(Llist)
Llist[1]

('a925ecc2-c223-5555-b66e-0b5b24122201',
 'ee9c03ac-4251-5042-ac85-986db9228a17')

Fixed!

### Graph 

https://networkx.org/documentation/stable/tutorial.html

In [47]:
# GRAPH

G = nx.Graph()

G.add_edges_from(Llist)

In [51]:
G.number_of_nodes(), G.number_of_edges()

(635, 11031)

This is less than the length of the linklist. Does it know that these eddes are weighted?

In [62]:
sum_node_deg = 0
for i in range(len(G.degree())):
    sum_node_deg = sum_node_deg + list(list(G.degree)[i])[1]
print(sum_node_deg) 

22062


**It knows. Yes!**

In [64]:
#Let's draw this

pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), 
                       node_color = values, node_size = 10)
nx.draw_networkx_edges(G, pos, edgelist=red_edges, edge_color='r', arrows=True)

plt.show()

NetworkXError: random_state_index is incorrect